# extract items and cik from 10-k

In [1]:
import re

In [2]:
info_reg_stack = {"comp_name": r'\s+company conformed name:\s+(.{3,100})',
                 "SIC": r'\s+standard industrial classification:\s+(.{3,100})',
                 'CIK': r'\s+central index key:\s+(\d{10})',
                 "date": r'\s+conformed period of report:\s+(\d{8})'}

In [3]:
def getInfo(filepath):
    import re
    result_dic = {}
    
    pattern = '(\s\sitem\s+\d+[a-zA-Z]?\.)'
    with open(filepath) as f:
        content = re.split(pattern, f.read().lower().replace('*',''))

    # extract items
    i = len(content)-1
    while i >= 0 :
        match = re.match(pattern,content[i])
        if match:
            key = match.group()[2:].replace(' ','')
            value = content[i+1].replace('\n','')
            
            if key not in result_dic: #and len(value) > 500:
                result_dic[key] = value
        i = i - 1
        
    # extract info
    for key, item in info_reg_stack.items():
        if re.search(item,content[0]):
            result_dic[key]=re.search(item,content[0]).group(1)
    
    return result_dic

In [4]:
def getInfo1(filepath):
    import re
    result_dic = {}
    
    pattern = '(\s\sitem\s+\d+[a-zA-Z]?\.)'
    with open(filepath) as f:
        
        content = re.split(pattern, f.read().lower().replace('_',''))

    # extract items
    i = len(content)-1
    while i >= 0 :
        match = re.match(pattern,content[i])
        if match:
            key = match.group()[2:].replace(' ','')
            value = content[i+1].replace('\n','')
            
            if key not in result_dic: #and len(value) > 500:
                result_dic[key] = value
        i = i - 1
        
    # extract info
    for key, item in info_reg_stack.items():
        if re.search(item,content[0]):
            result_dic[key]=re.search(item,content[0]).group(1)
    
    return result_dic

In [9]:
case = getInfo1('/Users/jiahuibi/Documents/projects/untitled folder/edgar_text/edgar/data/1750/0001047469-17-004528.txt')

In [10]:
case.keys()

dict_keys(['item16.', 'item15.', 'item14.', 'item13.', 'item12.', 'item11.', 'item10.', 'item9b.', 'item9a.', 'item9.', 'item8.', 'item7a.', 'item7.', 'item6.', 'item5.', 'item4.', 'item3.', 'item2.', 'item1b.', 'item1a.', 'item1.', 'comp_name', 'SIC', 'CIK', 'date'])

In [11]:
case['item7.']

"management's discussion and analysis of financial condition and results of operations21"

In [12]:
case['item1.']

'business2'

---------------------------------------

# get label

In [13]:
import pandas as pd
df = pd.read_csv('../../data/sec_index_complete_small_2018-01-22.csv')

In [14]:
df.head()

,gvkey,actual_10k_path_raw,fyear
0,1094,edgar/data/2034/0000002034-00-000006.txt,2000
1,1099,edgar/data/2070/0000941157-00-500008.txt,2000
2,12075,edgar/data/2491/0000950148-00-002040.txt,2000
3,1056,edgar/data/2601/0000932214-00-000160.txt,2000
4,1213,edgar/data/3000/0000003000-95-000003.txt,1994


In [15]:
len(df)

175699

In [16]:
cik_list = []
date_list = []

for path in list(df['actual_10k_path_raw']):
    your_path = '/Users/jiahuibi/Documents/projects/untitled folder/edgar_text/'
    final_path = your_path + path
    with open(final_path) as f:
        content = f.read().lower().replace('*','')
            
    try:
        CIK = re.search(r'\s+central index key:\s+(\d{10})',content).group(1)
        cik_list.append(CIK)
    except:
        cik_list.append(np.nan)
        pass
    
    try:
        date = re.search(r'\s+conformed period of report:\s+(\d{8})',content).group(1)
        date_list.append(date)
    except:
        date_list.append(np.nan)
        pass

In [17]:
df['CIK'] = cik_list
df['date'] = date_list

In [45]:
pd.to_datetime(df['date'])

0        2000-06-30
1        2000-06-30
2        2000-06-30
3        2000-06-30
4               NaT
5               NaT
6               NaT
7               NaT
8               NaT
9        2000-08-31
10       2000-07-31
11              NaT
12       2000-03-31
13       2000-06-03
14              NaT
15              NaT
16              NaT
17       2000-03-31
18              NaT
19              NaT
20       2000-04-30
21              NaT
22       2000-06-30
23       2000-06-30
24       2000-02-29
25       2000-01-29
26              NaT
27       2000-06-30
28              NaT
29              NaT
            ...    
175669   2017-09-30
175670   2017-09-30
175671   2017-10-31
175672   2017-10-29
175673   2017-09-30
175674   2017-06-30
175675   2017-08-31
175676   2017-08-31
175677   2017-09-30
175678   2017-09-30
175679   2017-08-31
175680   2017-09-30
175681   2017-09-30
175682   2017-09-30
175683   2017-09-03
175684   2017-08-31
175685   2017-09-30
175686   2017-07-31
175687   2017-09-30


In [18]:
df.head()

,gvkey,actual_10k_path_raw,fyear,CIK,date
0,1094,edgar/data/2034/0000002034-00-000006.txt,2000,0000002034,20000630
1,1099,edgar/data/2070/0000941157-00-500008.txt,2000,0000002070,20000630
2,12075,edgar/data/2491/0000950148-00-002040.txt,2000,0000002491,20000630
3,1056,edgar/data/2601/0000932214-00-000160.txt,2000,0000002601,20000630
4,1213,edgar/data/3000/0000003000-95-000003.txt,1994,NaN,NaN


In [19]:
df[df['CIK']=="N/A"]

,gvkey,actual_10k_path_raw,fyear,CIK,date


In [20]:
df['date']=pd.to_datetime(df['date'])

In [34]:
df.to_csv('../../data/index_add_date.csv')